https://github.com/huggingface/peft/blob/main/examples/image_classification/image_classification_peft_lora.ipynb

In [ ]:
# Code block to import own data in cloud
from google.colab import drive
drive.mount('/content/drive')
!rm -rf /content/hku_selfsup
!git clone -b main https://github.com/Colinnnnnm/hku_selfsup.git
!cp -r /content/drive/MyDrive/pretrained_model /content/pretrained_model
!cp -r /content/drive/MyDrive/data /content/hku_selfsup/data
!unzip -q /content/hku_selfsup/data/Market-1501-v15.09.15.zip -d /content/hku_selfsup/data
!mv /content/hku_selfsup/data/Market-1501-v15.09.15 /content/hku_selfsup/data/market1501
!unzip -q /content/hku_selfsup/data/market1501_grey.zip -d /content/hku_selfsup/data
!unzip -q /content/hku_selfsup/data/cuhk03.zip -d /content/hku_selfsup/data
!unzip -q /content/hku_selfsup/data/cuhk03_grey.zip -d /content/hku_selfsup/data
!cp /content/hku_selfsup/data/cuhk03/cuhk03_new_protocol_config_detected.mat /content/hku_selfsup/data/cuhk03_grey
!cp /content/hku_selfsup/data/cuhk03/cuhk03_new_protocol_config_labeled.mat /content/hku_selfsup/data/cuhk03_grey
!unzip -q /content/hku_selfsup/data/DukeMTMC-reID.zip -d /content/hku_selfsup/data
!mv /content/hku_selfsup/data/DukeMTMC-reID /content/hku_selfsup/data/duke
!unzip -q /content/hku_selfsup/data/duke_grey.zip -d /content/hku_selfsup/data
import torch
import numpy as np
import glob
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import argparse
import os
os.chdir("/content/hku_selfsup/TransReID-SSL/transreid_pytorch")
# Then interact with your own folder of data

Mounted at /content/drive
Cloning into 'hku_selfsup'...
remote: Enumerating objects: 3689, done.
remote: Counting objects: 100% (2418/2418), done.
remote: Compressing objects: 100% (1632/1632), done.
remote: Total 3689 (delta 921), reused 2207 (delta 741), pack-reused 1271
Receiving objects: 100% (3689/3689), 63.37 MiB | 18.97 MiB/s, done.
Resolving deltas: 100% (1271/1271), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import numpy as np

In [ ]:
!pip install transformers accelerate evaluate datasets git+https://github.com/huggingface/peft -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.6 MB/s eta 0:00:00


In [ ]:
import transformers
import accelerate
import peft

In [ ]:
print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"PEFT version: {peft.__version__}")

Transformers version: 4.34.1
Accelerate version: 0.23.0
PEFT version: 0.6.0.dev0


In [ ]:
ls /content/pretrained_model

dino_deitsmall16_pretrain.pth  dino_vitbase16_pretrain.pth     vit_base_ics_cfs_lup.pth
dino_deitsmall8_pretrain.pth   dino_vitbase8_pretrain.pth      vit_small_cfs_lup.pth
dinov2_vitb14_pretrain.pth     extracted_sam_vit_b_01ec64.pth  vit_small_cfs_lup_updated.pth
dinov2_vits14_pretrain.pth     extracted_sam_vit_h_4b8939.pth  vit_small_ics_cfs_lup.pth


In [ ]:
# config
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 64
model_checkpoint = '/content/drive/MyDrive/log/markett1501/dinov2_vits14_224x224/022/transformer_120.pth'

In [ ]:
model_checkpoint = '/content/pretrained_model/dinov2_vits14_pretrain.pth'
dino_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
type(dino_model)
dino_model.load_state_dict(torch.load(model_checkpoint, map_location=torch.device('cpu')))


Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


<All keys matched successfully>

In [ ]:
model_checkpoint = '/content/drive/MyDrive/log/markett1501/dinov2_vits14_224x224/022/transformer_120.pth'
dino_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
type(dino_model)
dino_model.load_state_dict(torch.load(model_checkpoint, map_location=torch.device('cpu')))


Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


RuntimeError: ignored

In [ ]:
model_checkpoint = '/content/drive/MyDrive/log/markett1501/dinov2_vits14_224x224/022/transformer_120.pth'
dino_model = torch.load('/content/hku_selfsup/dinov2/dinov2/models/vision_transformer.py', '/content/hku_selfsup/dinov2/dinov2/models/__init__.py')
type(dino_model)
dino_model.load_state_dict(torch.load(model_checkpoint, map_location=torch.device('cpu')))


UnpicklingError: ignored